In [0]:
import requests
import unicodedata
import re
import json
import time
import random
from bs4 import BeautifulSoup
from urllib.parse import urljoin

In [0]:
recipes             = []
links               = set() 
visited_links       = set()
page_links          = []
flag                = 'lospicchiodaglio'
counter             = 0
base_url            = "https://www.lospicchiodaglio.it"
url                 = "https://www.lospicchiodaglio.it/ricetta/besciamella" #seme 

In [0]:
#                                                      CRAWLING DEL SITO SPECIFICATO
#
#   Partendo dal link specificato estrapola tutti i link presenti nel codice html inserendoli in una lista dinamica.
#   Da questa lista verrà creato il set di link (evitando così la presenza di duplicati) che verrà unito al set totale contenente 
#   tutti i link incontrati ad ogni iterazione. Quelli visitati invece verranno inseriti nel set visited_links.
#   Il crawler estrarrà un link in modo random dal set dei link non ancora visitati per scegliere la sua prossima destinazione.
#   Le caratteristiche del codice varieranno in base alla struttura del sito target.


def surfer():
    
    global url
    global links
    global recipes
    global counter

    source_code = requests.get(url)
    plain_text = source_code.text
    soup = BeautifulSoup(plain_text, 'lxml')
    # test = soup.findAll('a')
    visited_links.add(url)

    for script in soup.find_all('script', type='application/ld+json'):
        #     if(soup.find('script', type='application/ld+json') != None):
        print("SCRIPT TROVATO!")
        js = json.loads(script.text, strict=False)
        if ("@type" in js and js["@type"] == "Recipe"):
            recipes.append(js["Name"])
            counter += 1
            print(js["Name"], "  N. ", counter)

    for link in soup.findAll('a'):
        href = link.get('href')
        if (href != url and href != '' and href != None):
            if not ((href.startswith('Javascript:')) or (href.startswith('javascript:')) or (href.startswith('/a/language ')) or (href.startswith('./+lospicchiodaglio')) or (href.startswith('http')) or (href.startswith('#')) or href.startswith('mailto:') or href.startswith('whatsapp')):
                href = urljoin(base_url, href)
                page_links.append(href)
            else:
                if flag in href and (href.startswith('http')) and not ((href.startswith('https://www.facebook'))or (href.startswith('https://it.pinterest'))or (href.startswith('https://plus.google'))or (href.startswith('https://twitter')) or (href.startswith('https://www.instagram'))):
                    page_links.append(href)
                

    # Modifica dell'insieme links
    links = links.union(set(page_links))

    # Sceglie, in modo random, un url dall'insieme dei link non visitati
    print("LINK VISITATO: ", url)
    url = random.sample(links.difference(visited_links), 1)[0]
    print("PROSSIMO LINK: ", url)
    print("------------------------------------------------------------------------------------------------------\n")

In [0]:
def run():

    # Termina solo se tutti i link sono stati visitati
    while (len(visited_links) <= len(links)):
        time.sleep(1)
        surfer()


In [0]:
run()

In [0]:
for recipe in recipes:
    rec = recipe.lower()
    recipe = rec

In [0]:
# salva il nome delle ricette in un file csv
csvFilename = "Dataset/RecipesSA_1.csv"  
csv = open(csvFilename, "w") 
colNames = "Recipes\n"  
csv.write(colNames)

In [0]:
for recipe in recipes:
    csv.write(recipe + "\n")